In [82]:
test_semantic_model_url = '/tmp/llm/semantic_model.txt' 
questions_url           = '/tmp/llm/questions.json'
SF                      = 0.1
max_attempts            = 1
TIMEOUT_SECONDS         = 180  
output_dir              = "/tmp/llm"
model0                  = "o3-mini"
model1                  = "qwen3-30b-a3b-2507"
model2                  = "gpt-oss-20b"

# Setup

In [83]:
import requests
import json
import duckdb
import pathlib
import time
import re
from   openai import AzureOpenAI, OpenAIError
import datetime
import os
import threading
import pandas as pd


In [84]:
if SF <1 :
 schema = f"DS{str(SF).replace('.', '_')}"
else :
 schema = f'DS{SF:02d}'
os.makedirs(output_dir, exist_ok=True)
db_path = output_dir +"/"+ schema +".duckdb"
if not pathlib.Path(db_path).exists():
    con = duckdb.connect(db_path)
    con.sql("SET memory_limit = '14GB' ")
    con.sql(f"CALL dsdgen(sf={SF})")
    con.close()
con = duckdb.connect()
con.sql(f""" attach '{db_path}' as ds(read_only) ; use ds """)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

In [85]:
def get_ai_response(user_message, LLM, provider, api_key=None, azure_endpoint=None, azure_api_version=None):
    system_prompt = ""
    try:
        github_response = requests.get(test_semantic_model_url)
        github_response.raise_for_status()
        system_prompt = github_response.text.strip()
    except requests.RequestException as e:
        try:
            with open(test_semantic_model_url, 'r', encoding='utf-8') as f:
                system_prompt = f.read().strip()
        except Exception as e2:
            print(f"Error loading system prompt from local path: {e2}")
            return f"Error fetching system prompt: {e2}"

    generated_text = None
    data = None

    if provider == "ollama":
        url = 'http://127.0.0.1:1234/v1/chat/completions'
        headers = {'Content-Type': 'application/json'}
        payload = {
            'model': LLM,
            'messages': [
                {'role': 'system', 'content': system_prompt},
                {'role': 'user', 'content': user_message}
            ],
            'stream': False
        }
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            data = response.json()
            generated_text = data.get('choices', [{}])[0].get('message', {}).get('content', '')
            if generated_text:
                generated_text = generated_text.replace("<think>", "").replace("</think>", "")
        except requests.RequestException as e:
            return f"Error with LM Studio API request: {e}"
        except (KeyError, IndexError, AttributeError) as e:
            return f"Unexpected response format from LM Studio: {e}\nFull response data: {data}"

    elif provider == "azure_foundry":
        if not azure_endpoint or not azure_api_version or not api_key:
            return "Azure provider requires 'azure_endpoint', 'azure_api_version', and 'api_key' parameters."

        try:
            client = AzureOpenAI(
                api_version=azure_api_version,
                azure_endpoint=azure_endpoint,
                api_key=api_key,
            )

            response = client.chat.completions.create(
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message}
                ],
                model=LLM,
            )

            generated_text = response.choices[0].message.content

        except Exception as e:
            return f"An error occurred with Azure OpenAI: {e}"



    else:
        return f"Unsupported provider for this test setup: {provider}. Supported providers are 'ollama' (LM Studio), 'azure_foundry', and 'gemini'."

    if not isinstance(generated_text, str):
         print(f"Generated text is not a string (type: {type(generated_text)}). Cannot clean.")
         if generated_text is None:
              return "Could not retrieve generated text from provider response."
         else:
              return f"Received unexpected output type from {provider}: {type(generated_text)}. Value: {generated_text}"

    # Enhanced text cleaning to extract SQL
    cleaned_text = generated_text
    
    # Simple approach: extract content after final channel message
    # Look for the pattern: <|channel|>final<|message|> and take everything after it
    final_message_match = re.search(r'<\|channel\|>final<\|message\|>(.*?)(?:<\|end\|>|$)', cleaned_text, re.DOTALL)
    if final_message_match:
        cleaned_text = final_message_match.group(1).strip()
    else:
        # Fallback: look for any final message pattern
        message_matches = re.findall(r'<\|message\|>(.*?)(?:<\|end\|>|$)', cleaned_text, re.DOTALL)
        if message_matches:
            cleaned_text = message_matches[-1].strip()  # Take the last message
        else:
            # Remove all chat template tokens if no specific pattern found
            cleaned_text = re.sub(r'<\|[^|]*\|>', '', cleaned_text).strip()
    
    # Remove any remaining template tokens
    cleaned_text = re.sub(r'<\|[^|]*\|>', '', cleaned_text).strip()
    
    # Remove thinking tags if present
    cleaned_text = re.sub(r'<think>.*?</think>', '', cleaned_text, flags=re.DOTALL | re.IGNORECASE).strip()
    
    # Clean up code blocks if present
    sql_in_blocks = re.findall(r'```(?:sql|duckdb)?\s*([\s\S]*?)\s*```', cleaned_text, flags=re.IGNORECASE)
    if sql_in_blocks:
        cleaned_text = sql_in_blocks[-1].strip()
    
    # Final cleanup
    cleaned_text = cleaned_text.replace("```sql", "").replace("```duckdb", "").replace("```", "").strip()
    
    # Simple extraction: return everything after the first '--' comment
    # This handles LM Studio's new format that puts SQL after explanatory text and comments
    double_dash_pos = cleaned_text.find('--')
    if double_dash_pos != -1:
        # Find the end of the comment line (newline after --)
        comment_end = cleaned_text.find('\n', double_dash_pos)
        if comment_end != -1:
            # Return everything after the comment line
            sql_after_comment = cleaned_text[comment_end + 1:].strip()
            if sql_after_comment:
                return sql_after_comment
    
    return cleaned_text

In [86]:
def execute_sql_with_retry(query, test_model, provider, api_key, azure_endpoint, azure_api_version):
    attempt = 1
    current_query = query.strip()

    while attempt <= max_attempts:
        con.sql("SET enable_progress_bar_print = false")
        con.sql("SET progress_bar_time = 0")

        result_container = {"result": None, "error": None}

        def query_thread():
            try:
                result_container["result"] = con.execute(current_query).fetchdf()
            except duckdb.InterruptException:
                result_container["error"] = f"Query interrupted after timeout of {TIMEOUT_SECONDS} seconds."
            except Exception as e:
                result_container["error"] = str(e)

        thread = threading.Thread(target=query_thread)
        thread.start()

        start_time = time.time()
        while thread.is_alive():
            elapsed = time.time() - start_time
            if elapsed > TIMEOUT_SECONDS:
                con.interrupt()
                thread.join()
                return f"Query execution timed out after {TIMEOUT_SECONDS} seconds.", attempt, "query runs forever"
            time.sleep(0.1)

        if result_container["error"]:
            error_message = result_container["error"].lower()
            if "syntax" not in error_message and "parser" not in error_message and "binder" not in error_message:
                return f"Non-syntax error: {result_container['error']}", attempt, current_query

            print(current_query)
            print(f"Attempt {attempt}/{max_attempts} failed with syntax error: {result_container['error']}")

            if attempt == max_attempts:
                return f"Max attempts reached. Last error: {result_container['error']}", attempt, current_query

            message = (
                f"The following SQL query has a syntax error: '{current_query}'.\n"
                f"Error message: {result_container['error']}\n"
                f"Please provide the corrected SQL query. Return only the corrected query without explanation."
            )

            corrected_query = get_ai_response(message, test_model, provider, api_key, azure_endpoint, azure_api_version)

            if corrected_query.startswith("Error"):
                return f"Failed to get corrected query : {corrected_query}", attempt, current_query

            current_query = corrected_query.strip()
            attempt += 1
        else:
            return result_container["result"], attempt, current_query

    return "Unexpected error or loop termination", attempt, current_query


In [87]:
def ask_question(questions, test_model,provider,api_key=None, azure_endpoint=None, azure_api_version=None):
    results_data = [] 
    for i, x in enumerate(questions):
        print(f"Question {i+1}: {x}") # Keep or remove print as needed
        start_time = time.time()
        sql_query_or_error = get_ai_response(x, test_model, provider, api_key, azure_endpoint, azure_api_version)
        print(sql_query_or_error) # Keep or remove print as needed
        query_result_data_json = [] # Initialize as an empty list for JSON result
        attempts_count = None
        error_details = None # Initialize error_details
        if sql_query_or_error is None or sql_query_or_error.startswith("Error"):
            # If get_ai_response returned an error or None, store the error string
            error_message = sql_query_or_error if sql_query_or_error is not None else "AI response was None"
            # print(f"Failed to get query from AI: {error_message}") # Keep or remove print
            # print("Execution: SKIPPED (AI error)") # Keep or remove print
            error_details = f"AI Error: {error_message}" # Store error details
            # query_result_data_json remains empty []
            result_row_count = 0 # Result count is 0 on error
        else:
            result_from_execution, attempts_count,query_returned = execute_sql_with_retry(sql_query_or_error, test_model,provider,api_key, azure_endpoint, azure_api_version)

            display(result_from_execution)
            is_successful = isinstance(result_from_execution, pd.DataFrame)

            if is_successful:
                print("Execution: SUCCESS") # Keep or remove print
                query_result_data_json = result_from_execution.to_dict('records')
                error_details = None # No error details on success
                result_row_count = len(result_from_execution) # Calculate row count
            else:
                print("Execution: FAILED") # Keep or remove print
                #query_result_data_json remains empty []
                error_details = f"Execution Error: {result_from_execution}" # Store error details
                result_row_count = 0 # Result count is 0 on failure


        end_time = time.time()
        duration = round(end_time - start_time, 2)
        print(f"\nExecution Time: {duration:.2f} seconds") # Keep or remove print
        print(f" ############################### ") # Keep or remove print
        results_data.append({
            "model" : test_model,
            "SF" : SF,
            "timestamp": timestamp,
            "nbr": i + 1,
            "question": x,
            "duration_s": duration, 
            "sql_query": query_returned, 
            "attempts": attempts_count,
            "result": query_result_data_json, 
            "result_count": result_row_count, 
            "error_details": error_details 
        })
    log_dir = output_dir +"/log"
    os.makedirs(log_dir, exist_ok=True)
    sanitized_model = re.sub(r'[\\/*?:"<>|]', '_', test_model)
    output_filename = f"{timestamp}_{sanitized_model}.json"
    output_path = os.path.join(log_dir, output_filename)
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(results_data, f, indent=4)
            f.flush()
            os.fsync(f.fileno())
        return f"Successfully processed {len(questions)} questions. Results saved to {output_path}"
    except IOError as e:
        return f"Error saving results to {output_path}: {e}"
    except Exception as e:
        return f"An unexpected error occurred during file saving: {e}"

In [88]:
def display_side_by_side(nbr, model1, model2):
    print(f"question {nbr} : " + duckdb.sql(f" select question from results_filtered where nbr = {nbr}  ").fetchone()[0])
    try:
        sql_query1 = duckdb.sql(f""" select sql_query from results_filtered where nbr = {nbr} and model = '{model1}' """).fetchone()[0]
        sql_query2 = duckdb.sql(f""" select sql_query from results_filtered where nbr = {nbr} and model = '{model2}' """).fetchone()[0]
        df1 = con.sql(sql_query1).df()        
        df2 = con.sql(sql_query2).df() 
        side_by_side = pd.concat([df1, df2], axis=1, keys=[model1, model2])
        display(side_by_side)
        # Print the SQL queries clearly labeled
        print(f"{model1} SQL Query:")
        print("--------------------")
        print(sql_query1)
        print("\n") # Add a newline for separation

        print(f"{model2} SQL Query:")
        print("--------------------")
        print(sql_query2)
        print("\n") # Add a newline for separation
    except Exception as e:
        print(f"Error executing query for nbr {nbr}: {e}")

In [89]:
# get the questions from GitHub, if error try local path
try:
    response = requests.get(questions_url)
    response.raise_for_status()
    questions = json.loads(response.text)
    print("Successfully retrieved questions from GitHub.")
except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
    print(f"Error retrieving from GitHub: Trying local path...")
    try:
        with open(questions_url, 'r', encoding='utf-8') as f:
            questions = json.load(f)
        print("Successfully loaded questions from local path.")
    except Exception as e2:
        print(f"Error loading questions from local path: {e2}")
        questions = []

Error retrieving from GitHub: Trying local path...
Successfully loaded questions from local path.


# Run Tests

In [90]:
%%time
#ask_question(questions,"o3-mini","azure_foundry",os.getenv("o3_mini_key"), os.getenv("llm_endpoint"),"2025-01-01-preview" )

CPU times: total: 0 ns
Wall time: 0 ns


In [91]:
#%%time
#ask_question(questions,model1,"ollama")

In [92]:
#%%time
# Uncomment to run the third model individually
#ask_question(questions,model2,"ollama")

# Model Comparison Results
Compare model1 vs model2 against the source of truth (model0)

In [93]:
# OPTIMIZED COMPARISON LOGIC - Much Faster!
# Pre-calculate sorted hashes to avoid repeated expensive operations
# FOCUS ON VALUES ONLY - IGNORE COLUMN NAMES COMPLETELY

# Load and prepare data with pre-calculated hashes focusing only on values
duckdb.sql(f"""
    CREATE OR REPLACE VIEW results_all AS
    SELECT *,
    cardinality(result[1]) as nbr_columns,
    [result_count , nbr_columns] as result_shape,
    -- Extract only values (ignore column names completely)
    flatten(array_transform(result, x -> json_extract(json(x), '$.*'))) as all_values,
    -- Hash of all values for exact comparison
    hash(flatten(array_transform(result, x -> json_extract(json(x), '$.*')))) as values_hash,
    -- Sorted hash for order-independent comparison (values only)
    hash(array_sort(flatten(array_transform(result, x -> json_extract(json(x), '$.*'))))) as sorted_values_hash
    FROM read_json_auto('{output_dir}/log/*.json') 
    WHERE model in ('{model0}', '{model1}', '{model2}') and SF = '{SF}' 
""")

duckdb.sql(f"""
    CREATE OR REPLACE TEMP TABLE results_filtered_three AS
    SELECT * FROM results_all  
    WHERE timestamp = (SELECT max(timestamp) FROM results_all r2 WHERE r2.model = results_all.model)
""")



# Enhanced comparison focusing exclusively on values with FIXED superset logic
comparison_table = duckdb.sql(f"""
    WITH source_results AS (
        SELECT nbr, 
               all_values as source_values,
               sorted_values_hash as source_sorted_hash,
               result_count as source_rows
        FROM results_filtered_three 
        WHERE model = '{model0}' AND error_details IS NULL
    ),
    model_scores AS (
        SELECT 
            r.model,
            r.nbr,
            r.result_count,
            r.duration_s,
            s.source_rows,
            CASE 
                WHEN r.error_details IS NOT NULL THEN 'error'
                -- Exact values match (same sorted hash) - ORDER AND VALUES IDENTICAL
                WHEN r.sorted_values_hash = s.source_sorted_hash THEN 'exact'
                -- IMPROVED SUPERSET LOGIC: If all source values exist in model result
                -- This handles cases where models return extra columns or calculations
                WHEN array_has_all(r.all_values, s.source_values) THEN 'superset'
                -- SUBSET: If model result is contained in source (shouldn't happen often)
                WHEN array_has_all(s.source_values, r.all_values) THEN 'subset'
                ELSE 'mismatch'
            END as match_type
        FROM results_filtered_three r
        LEFT JOIN source_results s ON r.nbr = s.nbr
        WHERE r.model IN ('{model1}', '{model2}')
    )
    SELECT 
        model,
        SUM(CASE WHEN match_type = 'exact' THEN 1 ELSE 0 END) as exact_matches,
        SUM(CASE WHEN match_type = 'superset' THEN 1 ELSE 0 END) as superset_matches,
        SUM(CASE WHEN match_type = 'subset' THEN 1 ELSE 0 END) as subset_matches,
        SUM(CASE WHEN match_type = 'mismatch' THEN 1 ELSE 0 END) as mismatches,
        SUM(CASE WHEN match_type = 'error' THEN 1 ELSE 0 END) as errors,
        SUM(CASE WHEN match_type IN ('exact', 'superset', 'subset') THEN 1 ELSE 0 END) as total_correct,
        ROUND(SUM(CASE WHEN match_type IN ('exact', 'superset', 'subset') THEN 1 ELSE 0 END) / 20.0 * 100, 1) as accuracy_percent,
        ROUND(SUM(duration_s), 1) as total_duration_seconds,
        ROUND(AVG(duration_s), 2) as avg_duration_per_question
    FROM model_scores
    GROUP BY model
    ORDER BY model
""").df()

print()
display(comparison_table)

,model,exact_matches,superset_matches,subset_matches,mismatches,errors,total_correct,accuracy_percent,total_duration_seconds,avg_duration_per_question
0,gpt-oss-20b,14.0,3.0,0.0,2.0,1.0,17.0,85.0,2071.1,103.55
1,qwen3-30b-a3b-2507,12.0,1.0,2.0,4.0,1.0,15.0,75.0,1651.0,82.55


In [95]:
# 🔥 COMPLETE TRANSPARENCY - SHOW ABSOLUTELY EVERYTHING 🔥


# Get EVERYTHING - all questions, all models, all match types
all_questions_analysis = duckdb.sql(f"""
    WITH source_results AS (
        SELECT nbr, 
               all_values as source_values,
               sorted_values_hash as source_sorted_hash,
               result_count as source_rows,
               sql_query as source_sql,
               question
        FROM results_filtered_three 
        WHERE model = '{model0}' AND error_details IS NULL
    ),
    model_scores AS (
        SELECT 
            r.model,
            r.nbr,
            r.question,
            r.sql_query as model_sql,
            r.result_count,
            r.error_details,
            s.source_rows,
            s.source_sql,
            CASE 
                WHEN r.error_details IS NOT NULL THEN 'error'
                WHEN r.sorted_values_hash = s.source_sorted_hash THEN 'exact'
                WHEN array_has_all(r.all_values, s.source_values) THEN 'superset'
                WHEN array_has_all(s.source_values, r.all_values) THEN 'subset'
                ELSE 'mismatch'
            END as match_type
        FROM results_filtered_three r
        LEFT JOIN source_results s ON r.nbr = s.nbr
        WHERE r.model IN ('{model1}', '{model2}')
    )
    SELECT nbr, question, model, model_sql, source_sql, result_count, source_rows, match_type, error_details
    FROM model_scores
    ORDER BY nbr, model
""").df()

print(f"📈 COMPLETE ANALYSIS SUMMARY:")
print(f"   Total Comparisons: {len(all_questions_analysis)}")

# Show comprehensive breakdown by match type
match_type_counts = all_questions_analysis['match_type'].value_counts()
for match_type, count in match_type_counts.items():
    emoji = {"exact": "✅", "superset": "⬆️", "subset": "⬇️", "mismatch": "❌", "error": "💥"}.get(match_type, "❓")
    percentage = (count / len(all_questions_analysis)) * 100
    print(f"   {emoji} {match_type}: {count} ({percentage:.1f}%)")
print()

from IPython.display import HTML
import html

# Process EVERY SINGLE QUESTION (1-20)
for question_num in range(1, 21):
    print(f"📋 QUESTION {question_num}")
    print("=" * 80)
    
    # Get question text from reference model
    try:
        question_text = duckdb.sql(f"""
            SELECT question FROM results_filtered_three 
            WHERE model = '{model0}' AND nbr = {question_num}
        """).fetchone()[0]
        print(f"❓ Question: {question_text}")
    except:
        print(f"❓ Question: [Could not retrieve question text]")
    
    # Get data for this question
    question_data = all_questions_analysis[all_questions_analysis['nbr'] == question_num]
    
    if len(question_data) == 0:
        print("⚠️ No data found for this question")
        print("=" * 80 + "\n")
        continue
    
    # Show match status for each model
    print(f"🏷️ Match Results:")
    for _, row in question_data.iterrows():
        model_name = row['model']
        match_type = row['match_type']
        emoji = {"exact": "✅", "superset": "⬆️", "subset": "⬇️", "mismatch": "❌", "error": "💥"}.get(match_type, "❓")
        
        if row['error_details']:
            print(f"   {emoji} {model_name}: {match_type} - {row['error_details']}")
        else:
            print(f"   {emoji} {model_name}: {match_type}")
    print()
    
    # Get reference result and SQL
    try:
        source_sql = duckdb.sql(f"""
            SELECT sql_query FROM results_filtered_three 
            WHERE model = '{model0}' AND nbr = {question_num} AND error_details IS NULL
        """).fetchone()[0]
        
        source_result = con.sql(source_sql).df()
        
        # Get results for all models
        all_models = [model0] + question_data['model'].tolist()
        all_results = [source_result]
        all_sqls = [source_sql]
        all_match_types = ['reference']
        
        # Get results for comparison models
        for _, row in question_data.iterrows():
            model_name = row['model']
            model_sql = row['model_sql']
            match_type = row['match_type']
            
            try:
                if row['error_details']:
                    # Add empty result for errors
                    all_results.append(pd.DataFrame({'Error': [row['error_details']]}))
                    all_sqls.append(model_sql)
                else:
                    model_result = con.sql(model_sql).df()
                    all_results.append(model_result)
                    all_sqls.append(model_sql)
                all_match_types.append(match_type)
            except Exception as e:
                all_results.append(pd.DataFrame({'Execution_Error': [str(e)]}))
                all_sqls.append(model_sql)
                all_match_types.append('execution_error')
        
        # Create comprehensive side-by-side comparison
        if len(all_results) > 1:
            max_rows = max(len(df) for df in all_results if len(df) > 0)
            if max_rows == 0:
                max_rows = 1
            
            padded_results = []
            for i, df in enumerate(all_results):
                if len(df) == 0:
                    padded_df = pd.DataFrame({'No_Data': ['Empty result']})
                elif len(df) < max_rows:
                    empty_rows = max_rows - len(df)
                    empty_df = pd.DataFrame([[None] * len(df.columns)] * empty_rows, 
                                          columns=df.columns)
                    padded_df = pd.concat([df, empty_df], ignore_index=True)
                else:
                    padded_df = df.copy()
                
                padded_results.append(padded_df)
            
            # Create model labels with match types
            model_labels = []
            for i, (model, match_type) in enumerate(zip(all_models, all_match_types)):
                if i == 0:
                    model_labels.append(f'{model} (Reference)')
                else:
                    emoji = {"exact": "✅", "superset": "⬆️", "subset": "⬇️", "mismatch": "❌", "error": "💥", "execution_error": "💥"}.get(match_type, "❓")
                    model_labels.append(f'{model} ({emoji} {match_type})')
            
            # Create multi-column comparison
            side_by_side = pd.concat(padded_results, 
                                   axis=1, 
                                   keys=model_labels[:len(padded_results)])
            
            print("📊 Complete Results Comparison:")
            display(side_by_side)
            
            # Create comprehensive HTML SQL comparison
            html_sections = []
            for i, (model_name, sql_query, match_type) in enumerate(zip(all_models, all_sqls, all_match_types)):
                if i == 0:
                    color = "#2e8b57"
                    bg_color = "#f8f9fa"
                    label = f"{model_name} SQL (Reference)"
                else:
                    emoji = {"exact": "✅", "superset": "⬆️", "subset": "⬇️", "mismatch": "❌", "error": "💥", "execution_error": "💥"}.get(match_type, "❓")
                    
                    if match_type == 'exact':
                        color = "#28a745"  # Green for exact
                        bg_color = "#f8fff8"
                    elif match_type in ['superset', 'subset']:
                        color = "#ff8c00"  # Orange for valid differences
                        bg_color = "#fff8dc"
                    else:
                        color = "#dc3545"  # Red for errors/mismatches
                        bg_color = "#fff5f5"
                    
                    label = f"{model_name} SQL ({emoji} {match_type})"
                
                section = f"""
                    <div style="flex: 1; border: 1px solid #ddd; padding: 10px; min-width: 300px; margin: 5px;">
                        <h4 style="color: {color}; margin-top: 0; margin-bottom: 10px;">{label}</h4>
                        <pre style="background-color: {bg_color}; padding: 10px; border-radius: 4px; white-space: pre-wrap; font-size: 11px; color: black; max-height: 300px; overflow-y: auto;">{html.escape(sql_query)}</pre>
                    </div>"""
                html_sections.append(section)
            
            html_content = f"""
            <div style="display: flex; gap: 10px; margin: 10px 0; flex-wrap: wrap; justify-content: space-around;">
                {''.join(html_sections)}
            </div>
            """
            
            print("🔧 Complete SQL Queries Comparison:")
            display(HTML(html_content))
            
    except Exception as e:
        print(f"❌ Error processing question {question_num}: {e}")
    
    print("=" * 80 + "\n")

print("🎯 COMPLETE ANALYSIS FINISHED - EVERY QUESTION SHOWN")
print("=" * 80)

📈 COMPLETE ANALYSIS SUMMARY:
   Total Comparisons: 40
   ✅ exact: 26 (65.0%)
   ❌ mismatch: 6 (15.0%)
   ⬆️ superset: 4 (10.0%)
   ⬇️ subset: 2 (5.0%)
   💥 error: 2 (5.0%)

📋 QUESTION 1
❓ Question: What is the overall total sales revenue?
🏷️ Match Results:
   ✅ gpt-oss-20b: exact
   ✅ qwen3-30b-a3b-2507: exact

📊 Complete Results Comparison:


,o3-mini (Reference),gpt-oss-20b (✅ exact),qwen3-30b-a3b-2507 (✅ exact)
,total_sales,total_sales_revenue,total_sales_revenue
0,5.143009e+08,5.143009e+08,5.143009e+08


🔧 Complete SQL Queries Comparison:



📋 QUESTION 2
❓ Question: What is the total number of items sold across all transactions?
🏷️ Match Results:
   ✅ gpt-oss-20b: exact
   ✅ qwen3-30b-a3b-2507: exact

📊 Complete Results Comparison:


,o3-mini (Reference),gpt-oss-20b (✅ exact),qwen3-30b-a3b-2507 (✅ exact)
,total_quantity,total_items_sold,total_items_sold
0,13939520.0,13939520.0,13939520.0


...

🔧 Complete SQL Queries Comparison:



📋 QUESTION 20
❓ Question: show all stores net sales , average net sales and percentage of net sales compared to average net sales for each store, ordered by percentage of net sales descending.
🏷️ Match Results:
   ✅ gpt-oss-20b: exact
   ✅ qwen3-30b-a3b-2507: exact

📊 Complete Results Comparison:


o3-mini (Reference)                                                  \
         s_store_name    net_sales avg_net_sales pct_net_sales_vs_avg   
0               ought  481787676.8   481787676.8                100.0   

  gpt-oss-20b (✅ exact)                                        \
             store_name    net_sales avg_net_sales pct_of_avg   
0                 ought  481787676.8   481787676.8      100.0   

  qwen3-30b-a3b-2507 (✅ exact)                                                  
                    store_name    net_sales avg_net_sales pct_of_avg_net_sales  
0                        ought  481787676.8   481787676.8                100.0

🔧 Complete SQL Queries Comparison:



🎯 COMPLETE ANALYSIS FINISHED - EVERY QUESTION SHOWN
